Isak Schwartz: 6 hours
Anton Forsberg: 6 hours
# Assignment 4

In [1]:
#Import scikit-learn dataset library
from sklearn import datasets

# Import train_test_split function
from sklearn.model_selection import train_test_split

#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

import numpy as np

import pandas as pd

import os

import re

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import MultinomialNB

from sklearn.naive_bayes import BernoulliNB

## 1: Preprocessing
We start by reading all the spam and ham mails from the given paths in the function createTokens(...). We then divide the mails into training and test data using the function train_test_split inside our custom function called printBAyesianAccuracies(). The split function distributes the mails into training and test data randomly with a distribution of 70% and 30% respectively. 



## 2 : python program


### 2.b : difference between multinomial and bernoulli naive bayes
The fundemental difference between the two is that binarized bernoulli used true or false for each word (or feature) in the mails to predict if it is spam or ham, whereas multinomial uses the actual amount of times each word occurs (say 0,1 or 34 etc.. ). 

For bernoulli this means that a word that occurs thousands of times can give the same weight when determining if a mail is spam or ham as a word that just occurs 1 time. For multinomial, the number of occurences for a word is more important.

In [2]:
def createTokens(pathsAndTargets,filter_Words, filter_Headers): # crates a list of tokens for all files in a given directory   
    files = []
    target = []
    for pathAndTarget in pathsAndTargets: # loop through all directories (ex: hard_ham and spam_2)
        
        for file in os.listdir(pathAndTarget[0]): # loop through all the files in the directories 
            filePath = (pathAndTarget[0] + "/" ) + file # file path to the specific file
            try:
                email = open(filePath,'r', encoding="utf-8").read()  #Read the file
                
                if(filter_Headers):  #remove haeders and footers (question 5) 
                    email = filetrFile(email)
                files.append(email) #add file to the dataset
                target.append(pathAndTarget[1]) # pathAndTarget[1] say if it is ham(1) or spam(0)
            except UnicodeDecodeError:
                pass
            
            
    # make vector from documents       
    vectorizer = CountVectorizer(analyzer='word')
    dataset = vectorizer.fit_transform(files).toarray()
    
    if(filter_Words):#filte away common and uncommon words (question 4)
        filterList = filterWords(vectorizer.get_feature_names(), dataset) #create list of common and uncommon words
        vectorizer = CountVectorizer(analyzer='word', stop_words = frozenset(filterList)) # make vector from documents  
        vectorizer._validate_vocabulary() 
        dataset = vectorizer.fit_transform(files).toarray() #create the data set
    elif(filter_Headers):
        vectorizer = CountVectorizer(analyzer='word', preprocessor = cleanhtml) # we give the preprocessor a function that removes html tags
        dataset = vectorizer.fit_transform(files).toarray() #create the data set
    
    return (dataset, target)

In [3]:
# function that removes headers and footers
def filetrFile(file):
    try: 
        index = file.index("<!-- ### footer ### -->") # remove the footer
        file = file[:index]
    except ValueError:
        pass
    try: 
        index = file.index("<!-- ### \header ### -->")+ len("<!-- ### \header ### -->") # remove header
        file = file[index:]
    except ValueError:
        pass

    try:
        index = file.index("\n\n") # remove header for non html emails (all (moast) files ended the email header with a blank row)
        file = file[index:]
    except ValueError:
        pass

    try:
        index = file.index("<html>")+ len("<html>") # remove header (this tag is the first thing occuring after the email header)
        file = file[index:]
    except ValueError:
        pass
    
    return file

In [4]:
# preprocessor function to remove html tags
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>') 
    cleantext = re.sub(cleanr, ' ', raw_html) #replace everything  surrounded by tags with a white space
    return cleantext

In [5]:
#function to filter out common and uncommn words
def filterWords(words, occurences):
    # give length equal to the data entries
    totalOccurences = np.zeros(len(occurences[0]))
    for mail in occurences:# loop throug all the email and count the total occurences of all the words
        for i in range (len(mail)):
            totalOccurences[i] += mail[i]
    # make filterList
    d = {'word': words, 'occurences': totalOccurences}
    df = pd.DataFrame(data=d)
    unfrequent = df.loc[df['occurences'] < 5]['word'].values # filter away words occuring less than 5 times
    frequent = df.loc[df['occurences'] > 1000]['word'].values # filter away words occuring more than 1000 times
 
    return np.concatenate((unfrequent , frequent)) #combine the two lists

In [6]:
# function where the data is split in to training and testing data, training the models and model evaluation is done
def printBayesianAccuracies(pathsAndTargets, filterWords, filterHeaders):
    dataset = createTokens(pathsAndTargets, filterWords, filterHeaders) # create the data sets
    ham = []
    hamTarget = []
    spam = []
    spamTarget = []
    
    for index in range(0, len(dataset[0])): #loop that sort the big dataset into the ham or spam dataset
        if(0 == (dataset[1][index])): # spam have the value 0
            spam.append(dataset[0][index]) #add to spam dataset
            spamTarget.append(0)
        else:
            ham.append(dataset[0][index]) #add to ham dataset
            hamTarget.append(1)
            
            
    # Split datasets into training set and test set (70-30)
    ham_train, ham_test, hamTarget_train, hamTarget_test = train_test_split(ham, hamTarget, test_size=0.3,random_state=0)
    spam_train, spam_test, spamTarget_train, spamTarget_test = train_test_split(spam, spamTarget, test_size=0.3,random_state=0)

    #create dataset that cointains all the data
    X_train = ham_train + spam_train
    y_train = hamTarget_train + spamTarget_train
    
    X_test = ham_test + spam_test
    y_test = hamTarget_test + spamTarget_test
    
    #Create a Naïve Bayes Classifiers
    classifiers = {(MultinomialNB(),"Multinomial naive bayes"), (BernoulliNB(binarize = 0), "Bernoulli naive bayes")}
    
    for classifier in classifiers:#loop through the two classifiers (Multinomial naive bayes and Bernoulli naive bayes)
        dataset = classifier[0]
        
        #Train the model using the training sets
        dataset.fit(X_train, y_train)

        #Predict the response for test dataset
        y_pred = dataset.predict(X_test)
        
        #Predict the response for ham dataset
        y_pred_ham = dataset.predict(ham_test)
        
        #Predict the response for spam dataset
        y_pred_spam = dataset.predict(spam_test)

        # Model Accuracy, how often is the classifier correct?
        print("Accuracy of " + classifier[1] + ": ",metrics.accuracy_score(y_test, y_pred))
        
        # Model Accuracy, how often is the classifier correct?
        print("Accuracy of" + classifier[1] + " on ham : ",metrics.accuracy_score(hamTarget_test, y_pred_ham))
        
        # Model Accuracy, how often is the classifier correct?
        print("Accuracy of" + classifier[1] + " on spam: ",metrics.accuracy_score(spamTarget_test, y_pred_spam))

In [7]:
# function to show a list of the occurences of every word
def wordTable(words, occurences):
    # give length equal to the data entries
    totalOccurences = np.zeros(len(occurences[0]))
    for mail in occurences:
        for i in range (len(mail)):
            totalOccurences[i] += mail[i]
    # make table
    d = {'word': words, 'occurences': totalOccurences}
    df = pd.DataFrame(data=d)
    df.sort_values(by=['occurences'], inplace = True)
    print(df)

## 3 : run program on easy and hard ham and include results

In [8]:
easyPaths = {("spam_2",0),("easy_ham_2",1)} # ({(directory, spam(0)),(directory })
hardPaths = {("spam_2",0),("hard_ham",1)}
    
print("spam_2 and easy_ham_2 Accuracies:")
printBayesianAccuracies(easyPaths, False, False)
print("spam_2 and hard_ham Accuracies:")
printBayesianAccuracies(hardPaths, False, False)

spam_2 and easy_ham_2 Accuracies:
Accuracy of Multinomial naive bayes:  0.9505851755526658
Accuracy ofMultinomial naive bayes on ham :  0.9948051948051948
Accuracy ofMultinomial naive bayes on spam:  0.90625
Accuracy of Bernoulli naive bayes:  0.9700910273081924
Accuracy ofBernoulli naive bayes on ham :  0.9948051948051948
Accuracy ofBernoulli naive bayes on spam:  0.9453125
spam_2 and hard_ham Accuracies:
Accuracy of Bernoulli naive bayes:  0.926829268292683
Accuracy ofBernoulli naive bayes on ham :  0.582089552238806
Accuracy ofBernoulli naive bayes on spam:  0.9869791666666666
Accuracy of Multinomial naive bayes:  0.9556541019955654
Accuracy ofMultinomial naive bayes on ham :  0.7611940298507462
Accuracy ofMultinomial naive bayes on spam:  0.9895833333333334


## 4 : filter out uncommon/common words
We print a list of words and the corresponding word count for each model we crate to see which words are common and uncommon. This is done in the function wordTables(...). But to actually choose which words to exclude from the model we use one numerical threshold which is how many times a word need to occur to be included, and another threshold which is the maximum amount of times a word can occur to be included. We chose these thresholds by tweaking them little by little and seeing if the final accuracies for the model improved. 

### 4.a : why is this useful? what words are common / uncommon?
It is useful to remove common words because they probably dont differ significantly between spam and ham. Meaning they are not a good indicator whther a mail is spam and ham. This also means these words can give other words less impact or weight when classifying mails, making classification less accurate overall. 

It is good to remove uncommon words because they can give strong associations to a certain type of mail even though they aren't typical for either spam or ham. For example, if a spam mail has the word "Saturn", which is a uncommon word that is not inherent to spam in any way, our model will think that all mails with the word saturn are spam (or at least make them way more likely to classidfied as spam). This is a problem because Saturn might be just as common in ham as spam.

By looking at the gnerated word count files we see that the 5 most common words for the model done on easy ham are : com, font, to, 3d and the. There are many other common words like is, of, by and with. The results are similar for hard ham.

The most uncommon words are g4q9dme14040, ntli, ntl4s72sppxz, developerworks, ntion and other seemingly random words that don't seem to be associated with either spam or ham.

### 4.b : filter using countVectorizer, how do results differ from part 3?

In [9]:
easyPaths = {("spam_2",0),("easy_ham_2",1)} 
hardPaths = {("spam_2",0),("hard_ham",1)}
    
print("spam_2 and easy_ham_2 Accuracies:")
printBayesianAccuracies(easyPaths, True, False)
print("spam_2 and hard_ham Accuracies:")
printBayesianAccuracies(hardPaths, True, False)

spam_2 and easy_ham_2 Accuracies:
Accuracy of Bernoulli naive bayes:  0.9856957087126138
Accuracy ofBernoulli naive bayes on ham :  0.9948051948051948
Accuracy ofBernoulli naive bayes on spam:  0.9765625
Accuracy of Multinomial naive bayes:  0.988296488946684
Accuracy ofMultinomial naive bayes on ham :  1.0
Accuracy ofMultinomial naive bayes on spam:  0.9765625
spam_2 and hard_ham Accuracies:
Accuracy of Multinomial naive bayes:  0.9866962305986696
Accuracy ofMultinomial naive bayes on ham :  0.9850746268656716
Accuracy ofMultinomial naive bayes on spam:  0.9869791666666666
Accuracy of Bernoulli naive bayes:  0.9290465631929047
Accuracy ofBernoulli naive bayes on ham :  0.746268656716418
Accuracy ofBernoulli naive bayes on spam:  0.9609375


## 5 : remove headers and footers, run program again

In [10]:
easyPaths = {("spam_2",0),("easy_ham_2",1)}
hardPaths = {("spam_2",0),("hard_ham",1)}
    
print("spam_2 and easy_ham_2 Accuracies:")
printBayesianAccuracies(easyPaths, False, True)
print("spam_2 and hard_ham Accuracies:")
printBayesianAccuracies(hardPaths, False, True)

spam_2 and easy_ham_2 Accuracies:
Accuracy of Multinomial naive bayes:  0.9687906371911573
Accuracy ofMultinomial naive bayes on ham :  0.9974025974025974
Accuracy ofMultinomial naive bayes on spam:  0.9401041666666666
Accuracy of Bernoulli naive bayes:  0.9427828348504551
Accuracy ofBernoulli naive bayes on ham :  0.9948051948051948
Accuracy ofBernoulli naive bayes on spam:  0.890625
spam_2 and hard_ham Accuracies:
Accuracy of Multinomial naive bayes:  0.9534368070953437
Accuracy ofMultinomial naive bayes on ham :  0.9552238805970149
Accuracy ofMultinomial naive bayes on spam:  0.953125
Accuracy of Bernoulli naive bayes:  0.9002217294900222
Accuracy ofBernoulli naive bayes on ham :  0.5522388059701493
Accuracy ofBernoulli naive bayes on spam:  0.9609375


### 5.a : do results improve from part 3 and 4?

### 5.b : how can data set split skew results? Remedies?
Our program always splits the spam and the ham the same way into training and test data, meaning 70% of the ham is used as training data and 70% of the spam is used as traing data. The alternative would be to merge all spam and ham into one data set and then split that into 70% training data and 30% test data. This solution won't impact the final distribution of spam and ham inside each set dramatically, it will still be around 70/70 and 30/30. 

Something to consider next is how many spam mails and ham mails there are overall. In our case we have about 1500 spam and 250 ham. This means that overall, mails are more likely to be classified as spam than ham COMPARED TO if there where say 1500 spam and 1500 ham GENERALLY. However, it is hard to say if this is good or bad, It might just be that 1500/250 is representative for the actual spam/ham ratio in the real world, meaning there shouldnt be the same amount of each in the training and test data either. So without further speculation, we can say that both of the proposed solutions in the paragraph above are fine and wouldn't skew the results more or less regardless of how many spam and ham mails there are in total.

The other way the split could be skewed is in terms of the actual distribution of training vs test data. If you divide it so you get too little training data (e.g 10% training data and 90 test data) the bayesian model can be too specialized to the few mails that appear in the training data. This means that the model would be bad at seeing general patterns in mails and at predicting whether a given mail is spam or ham. This would become apprant when looking at the accuracy score so you can try to tweak the distribution afterwards. If the training data is too big, you dont get enough test mails to verify it with, meaning it is hard to tell how good the model is since the test data will most likely on represent a small fraction of all mails. This maens the accuracy score might still be good but will still not tell you if it predicts any mail very well. This is a bigger problem than in the reversed case since you are less likely to notice something is wrong if you have a good accuracy.

To become more certain in the split distribution is good you can use cross validation. This means making similar tests such that every part of the data is used as training data in some validation, and testing data in some other validation. For example, splitting the data in 4 chunks means you can do 4 test, first use the first 25% of the data as test data and the rest as training data, for the next test use the first 26%-50% of the data as test data and the rest as training data and so forth... Lastly you take the average of each all the accuracy scores which will make it easier to tell if the model was good overall even if one single test might have given you a really high or low accuracy score. 

### 5.c : What would happen if training set was mostly spam, and test mostly ham
You would be way more likely to classify any given mail as spam. This is because most mails are spam in the training data so the model will most likely think most mails in the test data are spam as well. This can happen even if there are a few ham mails in the training data because the model will not become good enough to see small differences betweem spam and ham mails and general patterns for ham mails, meaning many ham mails will still get classified as spam. This is obviously bad since it would make such a spam filter put important mail in the clutter folder. 

